In [ ]:
# Imports

import pandas as pd
import itertools
import random
import numpy as np

def random_product(*args, repeat=1):
    "Random selection from itertools.product(*args, **kwds)"
    pools = [tuple(pool) for pool in args] * repeat
    return tuple(map(np.random.choice, pools))

In [ ]:
# Setup and data loading

data = pd.read_csv("data.csv").drop(columns=["Date","1st Participant Phone Number","2nd Participant Phone Number"])

appetizers_df = data.loc[data["Dish"].str.contains("Appetizer")]
main_df = data.loc[data["Dish"].str.contains("Main dish")]
dessert_df = data.loc[data["Dish"].str.contains("Dessert")]

appetizers_set = set(appetizers_df.index)
main_set = set(main_df.index)
dessert_set = set(dessert_df.index)

#events = itertools.product(appetizers,main_dish,dessert)
used_groups = []

In [ ]:
# Calculate appetizer groups
np.random.seed(1)

first_event = []
used_main = set()
used_dessert = set()
for pair in appetizers_set:
    group = random_product({pair},main_set-used_main,dessert_set-used_dessert)
    first_event.append(group)
    #used_groups.append(group)
    used_groups = used_groups + list(itertools.permutations(group))
    used_main.add(group[1])
    used_dessert.add(group[2])

    
appetizer_groups = []
for group in first_event:
    appetizer_groups.append(appetizers_df.loc[[group[0]]].append(main_df.loc[[group[1]]]).append(dessert_df.loc[[group[2]]]))
    

for g in appetizer_groups:
    display(g)

In [ ]:
# Calculate main_dish groups
np.random.seed(1)

second_event = []
used_appetizers = set()
used_dessert = set()
for pair in main_set:
    done = False
    while not done:
        group = random_product(appetizers_set-used_appetizers,{pair},dessert_set-used_dessert)
        if group not in used_groups:
            done = True
    second_event.append(group)
    #used_groups.append(group)
    used_groups = used_groups + list(itertools.permutations(group))
    used_appetizers.add(group[0])
    used_dessert.add(group[2])


main_groups = []
for group in second_event:
    main_groups.append(appetizers_df.loc[[group[0]]].append(main_df.loc[[group[1]]]).append(dessert_df.loc[[group[2]]]))
    
for g in main_groups:
    display(g)

In [ ]:
# Calculate dessert groups
np.random.seed(2)

third_event = []
used_appetizers = set()
used_main = set()
for pair in dessert_set:
    done = False
    while not done:
        group = random_product(appetizers_set-used_appetizers,main_set-used_main,{pair})
        if group not in used_groups:
            done = True
    third_event.append(group)
    #used_groups.append(group)
    used_groups = used_groups + list(itertools.permutations(group))
    used_appetizers.add(group[0])
    used_main.add(group[1])

dessert_groups = []
for group in third_event:
    dessert_groups.append(appetizers_df.loc[[group[0]]].append(main_df.loc[[group[1]]]).append(dessert_df.loc[[group[2]]]))
    
for g in dessert_groups:
    display(g)

In [ ]:
# Calculate people's route

address_map = {}
for person in data[" P1 Name"].append(data["P2 Name"]):
    for group in appetizer_groups:
        if group[" P1 Name"].append(group["P2 Name"]).str.contains(person).sum():
            appetizer_place = group["Coloc"].iloc[0]

    for group in main_groups:
        if group[" P1 Name"].append(group["P2 Name"]).str.contains(person).sum():
            main_place = group["Coloc"].iloc[1]

    for group in main_groups:
        if group[" P1 Name"].append(group["P2 Name"]).str.contains(person).sum():
            dessert_place = group["Coloc"].iloc[2]
    
    address_map[person] = (appetizer_place, main_place, dessert_place)

for p in address_map:
    print(p,address_map[p])
